# References
1. [Market Basket Analysis (Apriori) in Python](https://www.kaggle.com/yugagrawal95/market-basket-analysis-apriori-in-python)
2. [Market Basket Analysis - Exploring E-commerce data](https://www.kaggle.com/ostrowski/market-basket-analysis-exploring-e-commerce-data)
3. [Association Rules and the Apriori Algorithm: A Tutorial](https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html)

# Market Basket Analysis
## 1. Association rules
- 상품들이 서로 연관되어 있는지를 밝혀내는 기술
- 적용 분야
    - 유통: 구매 패턴을 파악해 매출을 높일 수 있다.
    - 의료: 어떤 증상이 함께 나타나는 경향이 있는지 파악하면 환자에게 더 좋은 진료를 해줄 수 있다. 여기서는 상품이 곧 증상이다.
- 상품 간의 관계를 측정하는 방법
    - **Support**: 해당 상품군이 얼마나 잘 팔리는지 나타냄.
        - 예시 1) 상품 X의 Support: 상품 {X}가 포함된 장바구니의 수 / 장바구니의 수
        - 예시 2) 상품 X, Y의 Support: 상품 {X, Y}가 포함된 장바구니의 수 / 장바구니의 수
    - **Confidence**: 상품 X를 구매할 때 상품 Y를 함께 구매하는 정도
        $$\frac{Support(X, Y)}{Support(X)}$$
        - Confidence는 X의 Support만 고려하고, Y의 Support는 고려하지 않기 때문에 관계를 잘못 측정할 수 있다. 예를 들어, Y가 베스트셀러라면 많은 장바구니에 들어가있을테고, 그럼 X와 Y의 실제 관계와는 상관없이 Confidence가 높게 나온다.
        - 이 단점을 보완하기 위한 것이 Lift이다.
    - **Lift**: 상품 X를 구매할 때 상품 Y를 함께 구매하는 정도. 단, Y가 얼마나 잘 팔리는 상품인지를 고려한다.
        $$\frac{Support(X, Y)}{Support(X) \times Support(Y)}$$
- 단점
    - 모든 상품 조합을 대상으로 Support를 계산해야 하는데, 상품이 $N$개일 때, $2^N-1$번 계산을 해야 한다.
## 2. Apriori Algorithm
- Association rules의 단점(고려해야할 경우의 수가 너무 많다)을 완화시키기 위한 알고리즘이다. '어떤 상품군의 Support가 작다면, 그것의 supersets의 Support도 작을 것이다.'는 가정을 하고 문제를 해결한다. 예를 들어, {pizza}의 Support가 낮다면 {pizza, soju}의 Support는 같거나 더 낮을 것이라는 것이다. 따라서 {pizza}의 Support를 계산한 후에 pizza가 포함되는 상품군의 Support는 계산할 필요가 없다.
- 작동 방법
    1. 하나의 상품만을 대상으로 Support를 계산한다.
    2. minimum support threshold를 만족하는 상품만 남겨둔다.
    3. 살아남은 상품을 조합하여 itemsets을 만든다.
    4. 새로운 itemsets이 나오지 않을 때 까지 2, 3을 반복한다.
- itemsets의 support를 계산해두면 confidence와 lift를 쉽게 계산할 수 있다.
- 단점
    - Computationally Expensive
        - 비록 Apriori Algorithm이 고려해야할 itemsets의 수는 줄였지만, 다루는 상품의 수가 많거나, minimum support threshold가 낮은 경우에는 여전히 고려해야할 경우의 수가 많다.
        - Hash Table과 같은 자료구조를 사용해서 계산량을 줄일 수 있다.
    - Spurious Associations
        - minimum support threshold가 낮은 경우, 상관이 있는지 의심되는 상품군이 발견될 수 있다.
        - 일반화 성능을 높이기 위해서 train, test set으로 나누고 테스트를 해볼 필요가 있다.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Loading libraries for python
import numpy as np # Linear algebra
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # Data visualization
import seaborn as sns # Advanced data visualization
import re # Regular expressions for advanced string selection
from mlxtend.frequent_patterns import apriori # Data pattern exploration
from mlxtend.frequent_patterns import association_rules # Association rules conversion
from mlxtend.preprocessing import TransactionEncoder
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', 
                 dtype={'InvoiceNo': str, 'StockCode': str, 'Description': str, 'Quantity': int, 'UnitPrice': float, 'CustomerID': str, 'Country': str}, 
                 encoding='ISO-8859-1',
                 parse_dates=['InvoiceDate'])
df.head()

In [ ]:
df.nunique().to_frame().transpose()

In [ ]:
df['StockCode'] = df['StockCode'].map(lambda x: x[:5])
df.nunique().to_frame().transpose()

In [ ]:
df = df[
    (df['Country'] == 'United Kingdom') &
    (~df['InvoiceNo'].str.startswith('C')) &
    (df['Quantity'] > 0) & 
    (df['UnitPrice'] > 0) & 
    (~df['StockCode'].str.isalpha()) &
    (~df['StockCode'].str.contains('BANK|C2|DCGS|gift'))
].reset_index(drop=True)
df.nunique().to_frame().transpose()

In [ ]:
df = df.groupby('InvoiceNo').filter(lambda x: len(x) >= 20)
df = df.groupby('StockCode').filter(lambda x: len(x) >= 20)
df.nunique().to_frame().transpose()

In [ ]:
cd_nm = df[['StockCode', 'Description']].drop_duplicates(subset=['StockCode'], keep='first')
cd_nm = dict(zip(cd_nm['StockCode'], cd_nm['Description']))

In [ ]:
basket = list(df.groupby('InvoiceNo')['StockCode'].agg(list))
basket[0]

In [ ]:
te = TransactionEncoder()
basket = te.fit(basket).transform(basket)
basket = pd.DataFrame(basket, columns = te.columns_)
basket.head()

In [ ]:
frequent_itemsets = apriori(basket, min_support=0.03, use_colnames=True)
frequent_itemsets

In [ ]:
rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules_mlxtend.head()

In [ ]:
rules_mlxtend['antecedents'] = rules_mlxtend['antecedents'].map(lambda x: [cd_nm[el] for el in x])
rules_mlxtend['consequents'] = rules_mlxtend['consequents'].map(lambda x: [cd_nm[el] for el in x])
rules_mlxtend.head()

In [ ]:
rules_mlxtend[ (rules_mlxtend['lift'] >= 4) & (rules_mlxtend['confidence'] >= 0.8) ][['antecedents', 'consequents']].head(50)

# 느낀점
- 데이터가 클 경우 (`InvoiceNo`가 많은 경우) minimum support threshold를 높게 주면 기준을 통과 못하는 것이 대다수. 그래서 threshold를 상당히 낮게 줘야 하는데, 이 경우 메모리가 부족할 수 있다.
    - 메모리 부족 문제를 해결하기 위해 `InvoiceNo`와 `StockCode`를 occurence 기준으로 필터링했다.
- {아이폰 -> 아이폰 케이스}처럼 찰떡같이 잘 어울리는 상품들이 나올줄 알았는데, 결과물을 보니 아예 같은 종류의 상품이 많이 나오는것 같다. 예를 들어, {PINK REGENCY TEACUP AND SAUCER -> GREEN REGENCY TEACUP AND SAUCER}는 색깔만 다를 뿐 같은 상품이다.